# Project 2 Machine Learning
## Road Segmentation Project.

This notebook has to  be runned in a Google Colab envioroment

MIN group. 

Mathilde Chaffard - Irene Vardabasso - Niccol`o Venturini Degli Esposti -

Import the requirements.txt file on Google Colab and after run the line below to install the necessary requirements.

In [ ]:
! pip install -r requirements.txt

Import the necessary libraries

In [ ]:
import numpy as np
from PIL import ImageFile, ImageEnhance, ImageFilter, Image
import sys, getopt
from torch import nn
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from sklearn import linear_model
from tqdm import tqdm
import torch
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from glob import glob
import matplotlib.image as mpimg
import os
sys.path.append('..')

Connect google colab to google drive to acces to the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #mount google drive

Define the parameters and the submission directory. Here the submission will be create in a google drive folder called submission, inside the EPFL folder, so on Google Drive has to be present a folder called EPFL, inside of this foldere has to be present a folder called submission and also has to be import the data, inside of EPFL folder, in a folder called data  

In [ ]:
DATA_PATH = "./drive/MyDrive/EPFL/data"
SUBMISSION_DIR = "./drive/MyDrive/EPFL/submission/"
SUBMISSION_PATH = "./drive/MyDrive/EPFL/submission/submission.csv"
PATCH_SIZE = 400 #value find with cross validation 400
BATCH_SIZE = 20
LR =  0.001 #value find with cross validation 0.001
MAX_ITER = 50
THRESHOLD = 0.25 #value find with cross validation 0.25
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TEST_SIZE = 0.2
K_FOLD = 5
SEED= 56
THRESHOLD_VALIDATION_VECTOR = [0.15,0.17,0.20, 0.25, 0.30, 0.35] # best found 0.25
LEARNING_RATE_VALIDATION_VECTOR = [0.0003, 0.0001, 0.001, 0.01]  # best found 0.001
PATCH_SIZE_VALIDATION_VECTOR = [80,400]  # best found 400



Import the pre-trained neural networks

In [ ]:
import segmentation_models_pytorch as smp
import torchvision.models as models

# import deeplabv3Plus
def get_deeplabplus(encoder_name = 'resnet34'):
    net = smp.DeepLabV3Plus(
    encoder_name=encoder_name,
    encoder_depth = 5,
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                     # model output channels (number of classes in your dataset)
    )

    return net

# import deeplabv3
def deeplab_model(encoder_name = 'resnet34'):
    net = smp.DeepLabV3(
        encoder_name=encoder_name,
        encoder_depth=5,
        encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1,  # model output channels (number of classes in your dataset)
    )

    return net

# import Deeplabv3Plus with resnet50
def get_resnet50(classes=1):
    #Create a DeepLabV3 model with ResNet50 backbone
    net = smp.DeepLabV3(
        encoder_name='resnet50',  # Use ResNet50 as the encoder
        encoder_depth=5,
        encoder_weights="imagenet",  # Use pre-trained weights for encoder initialization
        in_channels=3,  # Model input channels (3 for RGB images)
        classes=classes,  # Model output channels (1 for binary segmentation)
    )
    return net

Define helpers function

In [ ]:
#transform an image into a tensor, since when importing we are sure that we get np.arrays
def image_into_tensor(img:np.ndarray, device = None, divide = False):
    img = np.transpose(img,[2,0,1]) #change the order of the dimensions
    tensor = torch.Tensor(img) #transform the image into a tensor
    if divide:
        tensor = tensor / 255 #divide by 255 to have values between 0 and 1
    tensor.unsqueeze(0)
    if device is not None:
        tensor = tensor.to(device) #send the tensor to the device
    return tensor

#transform a mask into a tensor, since when importing we are sure that we get np.arrays
def mask_to_tensor(mask: np.ndarray, device = None):
    tensor = transforms.ToTensor()(mask) #transform the image into a tensor
    tensor = torch.round(tensor) #round the values to have only 0 and 1
    if device is not None:
        tensor = tensor.to(device) #send the tensor to the device
    return tensor[0,:,:][None,:,:]

#transform a tensor into a mask, since when importing we are sure that we get np.arrays
def transform_to_patch(pixels, threshold):
    m = np.mean(pixels) #compute the mean of the pixels
    if m  > threshold:
        return 1 #if the mean is above the threshold, we return 1
    else:
        return 0 #if the mean is below the threshold, we return 0

#transform an image into a np array, since when importing we are sure that we get np.arrays
def images_to_np_array(images):
    return np.array([np.array(img) for img in images]) #transform the images into np.arrays

#function to save a prediction changing the threshold to create the image for the report
def report(prediction, th):
    new_prediction = np.zeros(prediction.shape)
    new_prediction[prediction > th] = 150 #if the prediction is above the threshold, we put 150, so it is visible in the image
    return new_prediction

#take a predected image and translate it into a prediction on patches
def transform_prediction_to_patch(img, id, patch_size=16, step=16, th=0.25):
    prs = []
    ids = []
    for j in range(0,img.shape[1],step):
        for i in range(0, img.shape[0],step):
            threshold = th 
            prs.append(transform_to_patch(img[i:i+patch_size,j:j+patch_size],threshold)) #transform the image into a patch
            ids.append("{:03d}_{}_{}".format(id, j, i)) #create the id of the patch
    return prs, ids

#takes a np.array from images, and transform into PIL image (which is more usable to deal with chages on the image)
def PIL_Images_from_np_array(images):
    return list(map(Image.fromarray, images)) #transform the np.arrays into PIL images

#split between test and train set because we want to see the performance of our models
def split_data(x, y):
    if SEED is not None:
        np.random.seed(SEED) 
    indices = np.arange(len(x)) #create an array of indices
    np.random.shuffle(indices) #shuffle the indices

    split_point = int(len(indices) * (1 - TEST_SIZE)) #compute the split point

    train_x, train_y = np.array(x)[indices[:split_point]], np.array(y)[indices[:split_point]] #split the data for the train set
    test_x, test_y = np.array(x)[indices[split_point:]], np.array(y)[indices[split_point:]] #split the data for the test set

    return train_x, train_y, test_x, test_y

# load images
def load_image(infilename):
    data = mpimg.imread(infilename) #load the image
    return data

# Extract 6-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1)) #compute the mean of the image
    feat_v = np.var(img, axis=(0,1)) #compute the variance of the image
    feat = np.append(feat_m, feat_v) #append the mean and the variance
    return feat

# Compute features for each image patch
def value_to_class(v):
    df = np.sum(v) 
    if df > THRESHOLD:
        return 1 #if the sum of the values is above the threshold, we return 1
    else:
        return 0 #if the sum of the values is below the threshold, we return 0

Define the road segmentation configuration class

In [ ]:
class RoadSegmentationConfig():
    def __init__(self, divide_patches, normalize, augment, clahe):
        self.divide_patches= divide_patches
        self.norm=normalize
        self.aug=augment
        self.clahe=clahe

Define preprocessing and postprocessing function

In [ ]:
# normalize images between 0 and 1
def normalize_images(data):
    for i, image in enumerate(data):
        image = image / 255
        data[i] = (image - image.mean(axis=(0, 1), dtype='float64')) / (image.std(axis=(0, 1), dtype='float64')) #normalize the image
    return data

# augment images
def augment_images(data):
    augmented_images = []

    # Flip, rotate, blur and change brightness of the images
    for image in data:
        augmented_imgs = [
            image.transpose(Image.FLIP_LEFT_RIGHT),
            image.transpose(Image.ROTATE_90),
            image.transpose(Image.ROTATE_180),
            image.transpose(Image.ROTATE_270),
            image.transpose(Image.FLIP_TOP_BOTTOM),
            image.filter(ImageFilter.GaussianBlur(4)),
        ]

        color_shift = ImageEnhance.Color(image)
        augmented_imgs.append(color_shift.enhance(0.5)) #increase brightness

        for angle in range(10, 61, 10):
            augmented_imgs.append(image.rotate(angle, resample=Image.BICUBIC))
        augmented_images.extend(augmented_imgs)

    return augmented_images

# apply CLAHE to the dataset
def apply_clahe_to_dataset(data, clip_limit=2.0, tile_grid_size=(8, 8)):
    enhanced_data = []

    # Apply CLAHE to each image in the dataset
    for image in data:
        # Convert the image to grayscale if it's a color image
        if len(image.shape) == 3:  # Check if the image is color
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image

        # Create a CLAHE object
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

        # Apply CLAHE to the grayscale image
        clahe_image = clahe.apply(gray)

        # If the original image is color, merge the enhanced channel with the original channels
        if len(image.shape) == 3:
            clahe_image = cv2.cvtColor(clahe_image, cv2.COLOR_GRAY2BGR)

        enhanced_data.append(clahe_image)

    return np.array(enhanced_data)

# crop images into patches
def crop(image, width, height):
    res = []
    for i in range(0,image.shape[1],height):
        for j in range(0,image.shape[0],width):
            if len(image.shape) == 2: #if the image is grayscale
                res.append(image[j:j + width, i : i + height]) #crop the image
            else:
                res.append(image[j:j + width, i : i + height, :]) #crop the image
    return res


# aplly preprocessing to the data
def preprocess(data, gts, configuration:RoadSegmentationConfig, train: bool, ps):
    data = images_to_np_array(data)

    if train:
        gts = images_to_np_array(gts) #transform the groundtruths into np.arrays
    else:
        gts = None

    if configuration.norm:
        data = normalize_images(data) #normalize the images

    if not train:
        return data, None

    if configuration.aug:
        data = PIL_Images_from_np_array(data) #transform the images into PIL images

        if gts is not None:
            gts = PIL_Images_from_np_array(gts) #transform the groundtruths into PIL images

        augmented_images = augment_images(data) #augment the images

        if gts is not None:
            augmented_groundtruths = augment_images(gts) #augment the groundtruths
            gts.extend(augmented_groundtruths) #add the augmented groundtruths to the groundtruths

        data.extend(augmented_images) #add the augmented images to the images
        data = images_to_np_array(data) #transform the images into np.arrays

        if gts is not None:
            gts = images_to_np_array(gts) #transform the groundtruths into np.arrays

    if configuration.divide_patches and gts is not None: 
        patch_size = ps 
        data = [crop(image, patch_size,patch_size) for image in data] #crop the images
        data = np.asarray([data[i][j] for i in range(len(data)) for j in range(len(data[i]))]) #transform the images into np.arrays
        gts = [crop(image, patch_size, patch_size) for image in gts] #crop the groundtruths
        gts = np.asarray([gts[i][j] for i in range(len(gts)) for j in range(len(gts[i]))]) #transform the groundtruths into np.arrays
        data = images_to_np_array(data) #transform the images into np.arrays
        gts = images_to_np_array(gts) #transform the groundtruths into np.arrays

    if configuration.clahe:
        data= apply_clahe_to_dataset(data) #apply CLAHE to the images
        gts= apply_clahe_to_dataset(gts) #apply CLAHE to the groundtruths

    return data, gts

# load data
def load_data(path_data, path_gts, train : bool, device,configuration:RoadSegmentationConfig, ps):
    data = [Image.open(img) for img in path_data] #load the images
    
    gts = []
    if path_gts is not None:
        gts = [Image.open(gt) for gt in path_gts] #load the groundtruths

    if train:
        data, gts = preprocess(data, gts, configuration,True, ps) #apply preprocessing to the data and the groundtruths
    else:
        data, _ = preprocess(data, None, configuration, False, ps) #apply preprocessing to the data

    return data, gts

# apply postprocessing to the prediction
def postprocess(y):
    res = []
    y = np.array(y)
    sz = y[0][0].shape[0] #get the size of the image
    y = y.reshape((-1,6,sz,sz)) #reshape the prediction
    for images in y:
        one = np.fliplr(images[1]) #flip the image
        two = np.rot90(images[2], k=3) #rotate the image
        three = np.rot90(images[3], k=2) #rotate the image
        four = np.rot90(images[4],k=1) #rotate the image
        five = np.flipud(images[5]) #flip the image

        m = np.stack([images[0], one, two, three, four, five]) #stack the images
        m = np.mean(m,axis=0) #compute the mean of the images
        res.append(m) #append the mean to the result
    return res


Define the possible preprocessing step

In [ ]:
divide_patches = True
normalize = False
augment = True
training = True
validation = False
submmission = True
clahe=True
baseline = False

Define the dataset class

In [ ]:
class RoadSegmentationDataset(Dataset):
    def __init__(self, data_path, gt_path, configuration:RoadSegmentationConfig, train: bool, device = None, ps=PATCH_SIZE):
        self.train = train
        self.device = device
        imgs, gts = load_data(data_path,gt_path,train,device,configuration, ps) #load the data and the groundtruths
        divide = not configuration.norm #if we normalize the images, we don't divide by 255
        if gts is not None: 
            self.gt = [mask_to_tensor(gt,device) for gt in gts] #transform the groundtruths into tensors
        self.data = [image_into_tensor(img, device, divide = divide) for img in imgs] #transform the images into tensors

    def __len__(self):
        return len(self.data) #return the length of the data

    def __getitem__(self, index):
        if self.train:
            return self.data[index], self.gt[index] #return the data and the groundtruths
        else:
            return self.data[index] #return the data

Function to create the image to insert in the report

In [ ]:
def plot_prediction(prediction, dir):
    img = Image.fromarray(prediction)
    if img.mode != 'RGB':
        img = img.convert('RGB') #convert the image into RGB
    img_path = os.path.join(dir, "prediction.png") #create the path of the image
    img.save(img_path)  #save the image
    return img

Define the model class

In [ ]:
class RoadSegmentationModel(nn.Module):
    def __init__(self, device, lr = LR, th =THRESHOLD, max_iter = MAX_ITER, ps=PATCH_SIZE):
        super().__init__()
        self.device = device
        self.pre_trained_network = get_deeplabplus()
        self.criterion = nn.BCEWithLogitsLoss()
        self.pre_trained_network.to(self.device)
        self.th = th
        self.lr = lr
        self.max_iter = max_iter
        self.ps = ps
    
    #forward pass
    def forward(self, data):
        data_x = data[0].to(self.device) #data to the device
        return self.pre_trained_network(data_x) #return the prediction

    #train the model for one epoch
    def train_epoch(self, loader, optimizer):
        loss = []
        self.pre_trained_network.train() #set the model in training mode
        for batch in tqdm(loader): 
            pr = self.forward(batch) #predictions
            y = batch[1].to(self.device) #labels
            l = self.criterion(pr,y).to(self.device) #loss
            optimizer.zero_grad() #set the gradients to zero
            l.backward() #compute the gradients
            optimizer.step() #update the weights
            l = l.cpu().detach().numpy() #compute loss
            loss.append(l)
        return np.mean(loss)

    #test the model for one epoch
    def test_epoch(self, loader):
        self.pre_trained_network.eval() #set the model in evaluation mode
        loss = []
        with torch.no_grad():
            for batch in tqdm(loader):
                pr = self.forward(batch) #predictions
                y = batch[1].to(self.device) #labels
                l = self.criterion(pr,y).to(self.device) #loss
                l = l.cpu().detach().numpy() #compute loss
                loss.append(l)
        return np.mean(loss)

    #get the score of the model
    def get_score(self,loader, do_postprocessing=True):
        self.pre_trained_network.eval()
        prs = []
        ys = []
        first_predictions = self.make_prediction(loader,do_postprocessing) #make a prediction
        masks = loader.dataset.gt #get the groundtruths
        #transform the prediction into patches
        for pr, y in zip(first_predictions, masks):
            labels, _ = transform_prediction_to_patch(pr,1, patch_size =self.ps, th=self.th) #transform the prediction into patches
            prs.extend(labels) #extend the list of predictions
            y = y[0].cpu().detach().numpy() #get the mask
            real_labels, _ = transform_prediction_to_patch(y,1, patch_size =self.ps, th=self.th) #transform the mask into patches
            ys.extend(real_labels) #extend the list of labels
        return f1_score(ys,prs), accuracy_score(ys,prs)

    #make a prediction
    def make_prediction(self, loader, do_postprocessing):
        pr = []
        self.pre_trained_network.eval() #set the model in evaluation mode
        sigmoid = torch.nn.Sigmoid() #apply sigmoid to the output
        with torch.no_grad(): 
            for batch in tqdm(loader):
                p = sigmoid(self.pre_trained_network(batch.to(self.device))).cpu().detach().numpy() #apply sigmoid to the output
                pr.append(p[0][0]) #append the prediction
        if do_postprocessing:
            pr = postprocess(pr) #apply postprocessing
        return pr

    #train the model
    def train(self, train_loader, test_loader, evaluate, evaluate_loader, do_postprocessing):
        optimizer = torch.optim.Adam(self.pre_trained_network.parameters(), lr=self.lr) #define the optimizer
        losses = []
        test_losses = []
        accuracies = []
        f1s = []
        best_loss = {'loss': float('inf'), 'epoch': 0}
        i = 1
        while True:
            print("EPOCH " + str(i))
            l_train = self.train_epoch(train_loader, optimizer) #train the model
            print("epoch trained, now testing") 
            losses.append(l_train) #append the loss
            l_test = self.test_epoch(test_loader) #test the model
            test_losses.append(l_test) #append the loss
            if evaluate:
                print("Test Loss for epoch " + str(i) + " = " + str(l_test))
            else:
                print("Train Loss for epoch " + str(i) + " = " + str(l_test))
            f1 = 0
            acc = 0
            if evaluate:
                f1, acc = self.get_score(evaluate_loader, do_postprocessing) #get the score
                print("LOSS = " + str(l_test) + " F1 = " + str(f1) + " ACCURACY = " + str(acc))

            f1s.append(f1)
            accuracies.append(acc)

            if l_test < best_loss['loss']: 
                best_loss['loss'] = l_test
                best_loss['epoch'] = i

            if i == self.max_iter - 1:
                break

            i += 1
        
        # save the results into a dictionary
        results = {}
        results['train_loss'] = losses
        results['f1'] = f1s
        results['accuracy'] = accuracies
        results['test_loss'] = test_losses
        return results

    #create the submission file
    def submit(self, test_loader):

        prs = self.make_prediction(test_loader,False) #make a prediction
        img_ids = range(1,len(prs)+1) #get the ids of the images

        ret_ids = []
        ret_labels = []

        for pr, i in zip(prs,img_ids):
            #create the image thanks to the prediction, of the 6 image of the test set
            if i == 6:
                pred = report(pr, self.th) # save the prediction to create the image for the report
                plot_prediction(pred, SUBMISSION_DIR) #save the image

            labels, ids = transform_prediction_to_patch(pr,i,th=self.th) #transform the prediction into patches
            for label in labels:
                ret_labels.append(label) #extend the list of predictions
            for id in ids:
                ret_ids.append(id) #extend the list of ids

        pd.DataFrame({'id': ret_ids, 'prediction' : ret_labels}).to_csv(SUBMISSION_PATH,index=False) #create the submission file

Define the cross validation

In [ ]:
# build the k indices for cross validation
def build_k_indices(N, k_fold, seed = SEED):
    num_row = N
    interval = int(num_row / k_fold) #size of each fold
    np.random.seed(seed)
    indices = np.random.permutation(num_row) #random permutation of the indices
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)] #split the indices into k_fold
    return np.array(k_indices)

# cross validation step
def cross_validation_step(data, gts, k_indices, k, th, lr, ps):
    train_data = np.delete(data, k_indices[k], axis = 0) #get the train data
    train_gts = np.delete(gts, k_indices[k], axis = 0) #get the train labels
    test_x = []
    test_y = []
    for id in k_indices[k]:
        test_x.append(data[id]) #get the test data
        test_y.append(gts[id]) #get the test labels
    configuration=RoadSegmentationConfig (augment=True, normalize=True, divide_patches=True, clahe=True) #define the configuration

    train_set = RoadSegmentationDataset(train_data,train_gts,configuration, True, DEVICE, ps) #create the train set
    test_set = RoadSegmentationDataset(test_x,test_y,configuration,True,DEVICE, ps) #create the test set
    evaluation_dataset = RoadSegmentationDataset(test_x,test_y,configuration,False,DEVICE, ps) #create the evaluation set

    train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True) #create the train loader
    test_loader = DataLoader(test_set,BATCH_SIZE, shuffle=False) #create the test loader
    evaluation_loader = DataLoader(evaluation_dataset,1, shuffle=False) #create the evaluation loader

    model = RoadSegmentationModel(DEVICE,th=th,lr=lr,max_iter=10) #create the model
    results = model.train(train_loader, test_loader,True,evaluation_loader,False) #train the model
    f1_score = results["f1"] #get the f1 score
    return f1_score

# cross validation over the patch size
def validation_over_patch_size(data, gts, k_indices, k, ps):
    return cross_validation_step(data,gts,k_indices,k,THRESHOLD,LR, ps)

# cross validation over the threshold
def validation_over_threshold(data, gts, k_indices, k, threshold):
    return cross_validation_step(data,gts,k_indices,k,threshold,LR, PATCH_SIZE)

# cross validation over the learning rate
def validation_over_learning_rate(data, gts, k_indices, k, lr):
    return cross_validation_step(data,gts,k_indices,k,THRESHOLD,lr, PATCH_SIZE)

# cross validation
def cross_validation(data, gts, parameters, parameter_name, N, seed = SEED, k_fold = K_FOLD):
    k_indices = build_k_indices(N,k_fold,seed) #build the k indices
    best_performance = -1 
    optimal_parameter = -1

    for parameter in parameters:
        print("Trying " + str(parameter_name) + " = " + str(parameter))
        avg_performance = 0 #average performance
        performances = np.zeros(k_fold) #performances for each fold

        for k in range(k_fold):
            performance = 0
            if parameter_name == "threshold":
                performance = validation_over_threshold(data,gts,k_indices,k,parameter)[-1] #get the performance for the threshold
            elif parameter_name == "learning rate":
                performance = validation_over_learning_rate(data,gts,k_indices,k,parameter)[-1] #get the performance for the learning rate
            elif parameter_name == "patch size":
                performance = validation_over_patch_size(data,gts,k_indices,k,parameter)[-1] #get the performance for the patch size

            avg_performance = performance + avg_performance 
            performances[k] = performance #save the performance for the fold

        avg_performance = avg_performance / k_fold #compute the average performance

        print("Cross-Validation for " + parameter_name + " = " + str(parameter) + " with f1_score = " + str(avg_performance))
        if best_performance == -1 or avg_performance > best_performance: #if the performance is better than the best performance
            best_performance = avg_performance
            optimal_parameter = parameter

    print("Optimal Patameter for " + parameter_name + " = " + str(optimal_parameter))
    return optimal_parameter, best_performance

Define the logistic regression

In [ ]:
# load data for the logistic regression train
def load_data_log_reg_train(path_data, path_gts):
    files = os.listdir(path_data) #get the files
    n=len(files) #number of files
    imgs = [load_image(path_data + files[i]) for i in range(n)] #load the images
    gt_dir = path_gts  # ground truth directory
    print("Load " + str(n) + " images") 

    gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)] #load the ground truth images 

    img_patches = [crop(imgs[i], 16, 16) for i in range(n)] #crop the images into patches
    gt_patches = [crop(gt_imgs[i], 16, 16) for i in range(n)] #crop the ground truth images into patches

    # Linearize list of patches
    img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
    gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

    X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))]) #extract features
    Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))]) #get the labels
    return X, Y

# load data for the logistic regression test
def load_data_log_reg_test(path_data):
    files = os.listdir(path_data) #get the files
    n=len(files)
    imgs=[]
    for j in range (n):
        imgs.append(load_image(path_data + files[j] + "/" + files[j] + ".png")) #load the images 

    print("Load " + str(n) + " images")
    img_patches = [crop(imgs[i], 16, 16) for i in range(n)] #crop the images into patches

    # Linearize list of patches
    img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
    X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))]) #extract features

    return X


# logistic regression
def log_regression (x, y, x_test):

    logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced") #define the model
    logreg.fit(x, y) #train the model
    # Predict on the training set to print
    predict_test = logreg.predict(x_test)
    # Predict on the train (to verify logreg.predict working)
    predict=logreg.predict(x)

    # Get non-zeros in prediction and grountruth arrays
    predict_n = np.nonzero(predict)[0]
    y_n = np.nonzero(y)[0]

    TPR = len(list(set(y_n) & set(predict_n))) / float(len(predict)) #compute the true positive rate
    accuracy = accuracy_score(y, predict) #compute the accuracy
    f1 = f1_score(y, predict) #compute the f1 score
    print('True positive rate = ' + str(TPR))
    print('Accuracy score = ' + str(accuracy))
    print('F1 score = ' + str(f1))

    print (predict_test)
    return predict_test

# create the submission file for the logistic regression
def log_reg_submit (predict_test):
    img_ids = range(1,len(predict_test)+1) #get the ids of the images
    ret_ids = []
    ret_labels = []
    predict_test=np.asarray(predict_test) #transform the prediction into an array
    k=0
    j=0
    l=1
    for pr, i in zip(predict_test,img_ids): #create the submission file
        if (j>592):
          j=0
          k+=16
        if (k>592):
          j=0
          k=0
          l+=1

        id="{:03d}_{}_{}".format(l, k,j) #get the id of the image
        j+=16
        ret_labels.append(pr) 
        ret_ids.append(id)

    pd.DataFrame({'id': ret_ids, 'prediction' : ret_labels}).to_csv(SUBMISSION_PATH,index=False) #create the submission file

Define all the experiment

In [ ]:
EXPERIMENT = 4

if EXPERIMENT == 1:
    divide_patches = True
    normalize = False
    augment = False
    training = True
    submmission = True
    clahe=True
elif EXPERIMENT == 2:
    divide_patches = True
    normalize = True
    augment = False
    training = True
    submmission = True
    clahe=True
elif EXPERIMENT == 3:
    divide_patches = True
    normalize = False
    augment = True
    training = True
    submmission = True
    clahe=True

#EXPERIMENT 4 is the best one
elif EXPERIMENT == 4:
    divide_patches = True
    normalize = True
    augment = True
    training = True
    submmission = True
    clahe=False
    baseline = False
    # PATCH_SIZE = 400
    # LR = 0.001
    # THRESHOLD = 0.17

elif EXPERIMENT == 5:
    divide_patches = True
    normalize = True
    augment = True
    training = True
    submmission = True
    clahe=True
    baseline = False

#EXPERIMENT 6 is the baseline
if EXPERIMENT == 6:
    divide_patches = False
    normalize = False
    augment = False
    training = False
    submmission = False
    clahe = False
    baseline = True

Print the parameter used

In [ ]:
print(THRESHOLD)
print(LR)
print(PATCH_SIZE)
print(MAX_ITER)

Define and call the run function

In [ ]:
# define the run function
def run():
    torch.manual_seed(SEED)
    np.random.seed(SEED)

    configuration = RoadSegmentationConfig(augment=augment, normalize=normalize, divide_patches=divide_patches, clahe=clahe) #define the configuration

    data = sorted(glob(DATA_PATH + "/training/images/*.png")) #get the images
    gts = sorted(glob(DATA_PATH + "/training/groundtruth/*.png")) #get the ground truth images

    if baseline :
        submission_images = sorted(glob(DATA_PATH + "/test_set_images/*/*"), #get the submission images
        key = lambda x: int(x.split('/')[-2].split('_')[-1])) 
        x, y = load_data_log_reg_train(DATA_PATH + "/training/images/", DATA_PATH + "/training/groundtruth/") #load the data for the logistic regression
        x_test = load_data_log_reg_test(DATA_PATH + "/test_set_images/") #load the data for the logistic regression
        predict_regression=log_regression (x, y, x_test) #logistic regression
        log_reg_submit (predict_regression) #create the submission file
        return

    optimal_th = THRESHOLD
    optimal_lr = LR
    optimal_ps = PATCH_SIZE

    # cross validation
    if validation:
        print("Starting Cross Validation over Foreground Threshold")
        optimal_th,_ = cross_validation(data,gts,THRESHOLD_VALIDATION_VECTOR,"threshold",len(data))
        print("Starting Cross Validation over Learning Rate")
        optimal_lr,_ = cross_validation(data,gts,LEARNING_RATE_VALIDATION_VECTOR,"learning rate",len(data))
        print("Starting Cross Validation over Patch size ")
        optimal_ps,_ = cross_validation(data,gts,PATCH_SIZE_VALIDATION_VECTOR,"patch size",len(data))

    train_data, train_labels, test_data, test_labels = split_data(data,gts) #split the data between train and test set

    if submmission:
        # run training on full dataset
        train_data = data
        train_labels = gts

    train_set = RoadSegmentationDataset(train_data,train_labels,configuration, True, DEVICE, optimal_ps) #create the train set
    test_set = RoadSegmentationDataset(test_data,test_labels,configuration,True,DEVICE, optimal_ps) #create the test set
    evaluation_dataset = RoadSegmentationDataset(test_data,test_labels,configuration,False,DEVICE, optimal_ps) #create the evaluation set

    train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True) #create the train loader
    test_loader = DataLoader(test_set,BATCH_SIZE, shuffle=False)  #create the test loader
    evaluation_loader = DataLoader(evaluation_dataset,1, shuffle=False) #create the evaluation loader

    model = RoadSegmentationModel(DEVICE, th= optimal_th, lr = optimal_lr, max_iter = MAX_ITER, ps= optimal_ps) #create the model

    if training:
        if submmission:
            results = model.train(train_loader, test_loader,False,None,augment) #train the model
            f1 = None
            acc = None
        else: 
            results = model.train(train_loader, test_loader,True,evaluation_loader,False) #train the model
            f1 = results['f1'] #get the f1 score
            acc = results['accuracy'] #get the accuracy
        loss = results['train_loss'] #get the train loss
        test_loss = results['test_loss'] #get the test loss
        print("TRAINING LOSS = " + str(loss[len(loss)-1]))
        print("TEST LOSS = " + str(test_loss[len(test_loss)-1]))
    if submmission:
        submission_images = sorted(glob(DATA_PATH + "/test_set_images/*/*"), 
            key = lambda x: int(x.split('/')[-2].split('_')[-1])) #get the submission images 

        test_set = RoadSegmentationDataset(submission_images, None, configuration, False, DEVICE) #create the test set
        test_loader = DataLoader(test_set,1) #create the test loader

        model.submit(test_loader) #create the submission file

    return f1, acc

#call the run function and save the results (f1 score and accuracy)
f1,acc= run()

Plot the f1 score and the accuracy over the epochs, for the evaluation set, do not run this if submission is True

In [ ]:
import matplotlib.pyplot as plt

#plot the f1 score and the accuracy
def plot_f1(f1,acc):
    plt.plot(f1, label = 'f1')
    plt.plot(acc, label = 'accuracy')
    plt.xlabel('epoch')
    plt.title('F1 & accuracy')
    plt.legend()
    plt.show()

plot_f1(f1,acc)